# GPU-accelerated motion planning with cuRobo

In this notebook, we show you how to accelerate motion planning with [cuRobo](https://curobo.org).

Before continuing, make sure you have followed the cuRobo-specific installation instructions in the `airo-planner` README file.

## Preparing

We'll first download some necessary assets from Nvidia.

In [ ]:
! wget https://raw.githubusercontent.com/NVlabs/curobo/refs/heads/main/src/curobo/content/assets/robot/ur_description/ur5e.urdf -O /tmp/ur5e.urdf
! wget https://raw.githubusercontent.com/NVlabs/curobo/refs/heads/main/src/curobo/content/configs/robot/ur5e.yml -O /tmp/ur5e.yml
! wget https://raw.githubusercontent.com/NVlabs/curobo/refs/heads/main/src/curobo/content/configs/world/collision_table.yml -O /tmp/collision_table.yml

## Creating the environment

For creating the world, you can use a world file containing all meshes, cuboids, etcetera. Or, you can create your world programmatically. For more information, refer to [the cuRobo documentation](https://curobo.org/get_started/2c_world_collision.html).

Here, we'll use a world file.

In [ ]:
robot_file = "/tmp/ur5e.yml"
world_file = "/tmp/collision_table.yml"

In [ ]:
from airo_planner.curobo.single_arm_curobo_planner import SingleArmCuroboPlanner
import numpy as np
import time

## Setting up the planner

We can set up the cuRobo planner with a single line.

In [ ]:
planner = SingleArmCuroboPlanner(robot_file, world_file)

## Planning in joint state

We'll move from a start configuration to a goal configuration.

In [ ]:
q_start = np.deg2rad([0, -90, 90, -90, -90, 0])
q_goal = np.deg2rad([90, -180, 90, 180, -110, 0])

In [ ]:
trajectory = planner.plan_to_joint_configuration(q_start, q_goal)

Let's visualize the trajectory in rerun.
Open a new terminal, and start rerun: `uv run rerun`. Then, run the next cell.

In [ ]:
from airo_planner.curobo.rerun_visualizer import CuRoboRerunVisualizer
viz = CuRoboRerunVisualizer("curobo", "ur5e")

viz.log_curobo_state(planner.motion_gen.world_model)

t = 0
for i, q in enumerate(trajectory.path.positions):
    viz.log_robot_configuration(q)
    time.sleep(trajectory.times[i] - t)
    t = trajectory.times[i]

## Planning to an EEF pose

To plan in EEF state, we'll have to create a TCP pose. We'll use the forward kinematics method as a way to obtain this.

In [ ]:
tcp_pose = planner.forward_kinematics(q_goal)
q_goal_hat = planner.inverse_kinematics(tcp_pose)  # Just as an example...

In [ ]:
trajectory = planner.plan_to_tcp_pose(q_start, tcp_pose)

In [ ]:
t = 0
for i, q in enumerate(trajectory.path.positions):
    viz.log_robot_configuration(q)
    time.sleep(trajectory.times[i] - t)
    t = trajectory.times[i]

## Adding obstacles

What's nice about cuRobo is that it allows us to add obstacles at runtime - unlike Drake, we don't need to rebuild the scene (which can be a costly operation).

In [ ]:
from curobo.geom.types import *

current_obstacles = planner.get_collider_cuboids()
current_obstacles.append(Cuboid(
    name="will_cause_collision",
    pose=[0.0, 0.0, 1.0, 0.043, -0.471, 0.284, 0.834],
    dims=[0.1, 0.5, 0.1],
))
planner.set_collider_cuboids(current_obstacles)

In [ ]:
trajectory = planner.plan_to_tcp_pose(q_start, tcp_pose)

In [ ]:
viz.log_curobo_state(planner.motion_gen.world_model)
t = 0
for i, q in enumerate(trajectory.path.positions):
    viz.log_robot_configuration(q)
    viz.log_robot_collision_spheres(planner.motion_gen.kinematics, q)
    time.sleep(trajectory.times[i] - t)
    t = trajectory.times[i]

## Batched planning

Another useful feature of cuRobo is batched planning. If you need to compute multiple trajectories, this can drastically speed up the computations.

First, we plan to TCP poses.

In [ ]:
q_starts = [q_start]
goal_tcps = [tcp_pose]
for i in range(1, 4):
    tcp_pose[2, 3] += 0.02 * i
    goal_tcps.append(tcp_pose)
    q_starts.append(planner.inverse_kinematics(tcp_pose).squeeze().cpu().numpy())
trajectories = planner.plan_to_tcp_poses_batched(q_starts, goal_tcps)
for trajectory in trajectories:
    t = 0
    for i, q in enumerate(trajectory.path.positions):
        viz.log_robot_configuration(q)
        viz.log_robot_collision_spheres(planner.motion_gen.kinematics, q)
        time.sleep(trajectory.times[i] - t)
        t = trajectory.times[i]

Now, we plan to joint configurations. Internally, this computes forward kinematics from goal poses since cuRobo only supports batched planning to poses (and not joint configurations).

In [ ]:
q_starts = [q_start]
q_goals = [q_goal]
for i in range(1, 8):
    qs_new = q_goals[i - 1].copy()
    qg_new = qs_new.copy()
    qg_new[0] += np.pi / 8
    q_starts.append(qs_new)
    q_goals.append(qg_new)
trajectories = planner.plan_to_joint_configurations_batched(q_starts, q_goals)
for trajectory in trajectories:
    t = 0
    for i, q in enumerate(trajectory.path.positions):
        viz.log_robot_configuration(q)
        viz.log_robot_collision_spheres(planner.motion_gen.kinematics, q)
        time.sleep(trajectory.times[i] - t)
        t = trajectory.times[i]